CG-Net.ipynb 
* Created by: Carter Lyons

Contains:
* Deep neural network implementation of Compound Gaussian Network (CG-Net) form by applying algorithm unrolling to the Compound Gaussian Least Squares (CG-LS) iterative algorithm. Estimates signal representation coefficients from linear measurements.
* For more information reference:
    1. Lyons, C., Raj, R. G., & Cheney, M. (2023). "A Compound Gaussian Network for Solving Linear Inverse Problems". arXiv preprint arXiv:2305.11120.
    2. Lyons C., Raj R. G., and Cheney M. (2022). "CG-Net: A Compound Gaussian Prior Based Unrolled Imaging Network," in *2022 IEEE Asia-Pacific Signal and Information Processing Association Annual Summit and Conference*, pp. 623-629.

Package Requirements (in addition to those required in Initialize.py):
* Tensorflow 2.5.0, pandas, matplotlib

In [12]:
# Import necessary python packages
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

# Import necessary additional code files
import Initialize
import LossFunctions
from CGNet import CGNet

# Create Network Structure 

In [18]:
# Choose network parameters
img_size = 32  # Height and width of input images 
style = 'tridiag'  # Format of PD matrix in z quadratic descent direction. Options: 'tridiag', 'full'
angles = 15  # Number of uniformly spaced angles for Radon transform
noise = 60  # SNR of measurements (in dB)
K = 20  # Number of iterations to unroll
J = 1  # Number of steepest descent updates of z for each interation to unroll
scale = 1  # Parameter to scale the input measurements by
path_to_save = r'path_to_desired_save_folder'  # Path to save weights trained by the network.

n = img_size**2  # Signal size
B = Initialize.create_measurements('barbara.png', [[154, 154+img_size],[64,64+img_size]], import_phi = (False, [], 'bior1.1'), import_psi = (False, [], angles))  # From Initialize.py: Create Radon transform matrix. Create biorthogonal wavelet matrix. Use these matrices to form measurements of input 'Barbara' image
A = tf.constant(B.Psi@B.Phi, dtype = float)  # Measurement matrix: A = Psi*Phi (set dtype to float32)

In [4]:
# Network structure
model = CGNet(K, J, A, style, 0.1, kappa = 2, eta = 0.5, eps = 1e-3, method = 'g', normalize = True)
model = model.call()
loss_fnc = LossFunctions.SSIM_loss(B.Phi, img_size, scale)

In [7]:
num_training_samples = 20
model.load_weights(r'Weights\CGNet_{}angles_{}dB_{}_samples.h5'.format(angles, noise, num_training_samples))  # Load network variables (if applicable)

# Read in Testing Data

In [8]:
path_to_test_data = r'Datasets\200_32x32_{}angles_{}dB_test_data.csv'.format(angles, noise)  # Path to testing dataset
test_data = pd.read_csv(path_to_test_data)  # Read in testing dataset
test_data.drop(test_data.columns[0], axis=1, inplace=True)  # Drops first column of dataset, which is a numbering of the rows

# Average Quality Metrics on Test Data

In [ ]:
import Validate
V = Validate.AverageMetrics(B.Phi, img_size, scale, power = 1, normalize = True)  # From Validate.py: Used to calculate SSIM, PSNR, MSE, and MAE between estiamted and actual signals

test_data_size = len(test_data)
num_batches = int(np.ceil(test_data_size/TestBatchSize))
run_time = []
for batch in range(num_batches):
    y = scale*np.array(test_data.iloc[batch*TestBatchSize:min([(batch+1)*TestBatchSize,test_data_size])])
    c_act, y = y[:,np.shape(B.Psi)[0]:], y[:,:np.shape(B.Psi)[0]]
    start = time.time()
    c_est = model(y, training = False)
    run_time.append((time.time()-start)/np.shape(y)[0])
    V.call(c_act, c_est)  # Calculate SSIM, PSNR, MSE, and MAE between estiamted and actual signal

V.data['TIME'] = run_time    
data = []
for entry in V.data:
    data.append([entry, np.mean(V.data[entry]), np.var(V.data[entry]), 2.576*np.sqrt(np.var(V.data[entry])/test_data_size)])  # Calculates average over all test data, variance over all test data, and 99% confidence interval
    print(entry+' AVE = {:0.3e} '.format(np.mean(V.data[entry]))+entry+' VAR = {:0.3e}'.format(np.var(V.data[entry])))
r = pd.DataFrame(data)
r.to_csv(r'{}_TestResults.csv'.format(path_to_save))  # Save test results

# Visual Test Images

In [24]:
y = np.transpose(B.add_noise(noise, B.y))
est = model(y, training=False)
c_est = np.array(tf.transpose(est))
I_est = B.Phi@c_est

In [ ]:
I_act = B.I
plt.subplot(1, 2, 1)
plt.imshow(np.reshape(I_est, (img_size, img_size)), cmap = 'gray')  # Display estimated image

plt.subplot(1, 2, 2)
plt.imshow(np.reshape(I_act, (img_size, img_size)), cmap = 'gray')  # Display actual image

plt.show()

In [9]:
start = 0
num_images = 10
y = scale*np.array(test_data.iloc[start:start+num_images])
c_act = y[:,np.shape(B.Psi)[0]:]
y = y[:,:np.shape(B.Psi)[0]]
est = model(y, training=False)
c_est = np.array(tf.transpose(est))
I_est = np.array(B.Phi@c_est)

In [ ]:
i = 9  # Anything between 'start' and 'start+num_images-1'
I_act = B.Phi@np.transpose(c_act[i])
plt.subplot(1, 2, 1)
plt.imshow(np.reshape(I_est[:,i], (img_size, img_size)), cmap = 'gray')  # Display estimated image

plt.subplot(1, 2, 2)
plt.imshow(np.reshape(I_act, (img_size, img_size)), cmap = 'gray')  # Display actual image

plt.show()